In [6]:
import glob
import gensim
import pyLDAvis
import Stemmer

import pandas as pd
import numpy as np
import pyLDAvis.gensim as gensimvis

from gensim.corpora.dictionary import Dictionary
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

from nltk.tokenize import RegexpTokenizer

from tqdm import tqdm_notebook

In [7]:
df_aca = pd.read_csv('../academia.csv', encoding='utf-8-sig', usecols=['Combined'])
df_aca

,Combined
0,the nature of economic development
1,commodity exports and the adding up problem in...
2,japanization context and culture in the indon...
3,industrial reorganization in developing countr...
4,fiscal adjustment and growth in cote d ivoire
5,policy lessons from lao and hmong women in tha...
6,brazilian privatization in the 1990s
7,egalitarianism subsistence provision and work ...
8,labor market implications of privatization in ...
9,the changing position of women in russian indu...


In [8]:
tokenizer = RegexpTokenizer(r'\w+')
en_stemmer = Stemmer.Stemmer('en')

docs = df_aca['Combined'].values

for i in tqdm_notebook(range(len(docs))):
    docs[i] = tokenizer.tokenize(docs[i]) 
    
docs = [[token for token in doc if not token.isnumeric()] for doc in tqdm_notebook(docs)]
docs = [[token for token in doc if len(token) > 1] for doc in tqdm_notebook(docs)]
docs = [en_stemmer.stemWords(doc) for doc in tqdm_notebook(docs)]

In [9]:
grams = []
bigram = Phrases(docs, min_count=15)
for i in tqdm_notebook(range(len(docs))):
    tmp = []
    for token in bigram[docs[i]]:
        if '_' in token:
            tmp.append(token)
        
    grams.append(tmp)

/usr/local/lib/python3.4/dist-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [10]:
dictionary = Dictionary(grams)
dictionary.filter_extremes(no_below=10, no_above=0.5)

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in tqdm_notebook(grams)]

In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 12742
Number of documents: 743100


In [13]:
temp = dictionary[0]
id2word = dictionary.id2token

model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, passes=1, workers=6)

In [14]:
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.4/dist-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]
